# Stream processing

## 1. Message queue

In this section we will introduce Apache Kafka ( https://kafka.apache.org/ ). Kafka is a distributed message broker were you can subscribe for a topic and receive a stream of messages. You will also see how to publish messages. Let's start with the initial setup.

You will be using the pykafka library ( http://pykafka.readthedocs.io/en/latest/ ). It should be already installed on the ICCluster, but you may need to install it locally if you work on your own computer. 

In [ ]:
from pykafka import KafkaClient
import getpass

ZOOKEEPER_QUORUM = 'iccluster042.iccluster.epfl.ch:2181,iccluster054.iccluster.epfl.ch:2181,iccluster078.iccluster.epfl.ch:2181'

username = getpass.getuser()

client = KafkaClient(zookeeper_hosts=ZOOKEEPER_QUORUM)

**1.a** Look at the pykafka documentation and list the topics currently available:

In [ ]:
#TODO: print the topics
client.<FILL>

As you can see there are already some topics there and unfortunately no separation per user. So, be respectful with the work of your colleagues and with the infrastructure we put in place. More specifically, never write in a topic that you didn't create yourself or your teammate (if not expressly asked to). Be also carefull not to create infinite loops by publishing in the same topic you subscribed, without filtering or exit conditions.

**1.b** That being said, let's create our own topic. Don't forget to make it unique with your username.

In [ ]:
name = ('test-topic-' + username).encode()
#TODO: create topic and get a handle
tt=<FILL>

Kafka is configured to autocreate topics if they don't exist a priori. You can check in the list of topics, it should be there now. 

**1.c** To write a message to the topic you need to instanciate a Producer. For simplicity and as we won't deal with very large (or very fast) data, you can use a synchronous publisher. Then, send a series of 10 different messages (for example "Hello world {i}!" with i from 1 to 10). You might need to encode the string to utf-8 before sending:

In [ ]:
with tt.get_sync_producer() as producer:
    for i in range(10):
        #TODO produce Hello world i

**1.d** Now let's check what is in our topic and subscribe to it. Again, for simplicity reason, you can use the basic ```get_simple_consumer```. Notice that the reading loop never finishes, streams are infinite! You may want to catch the KeyboardInterrupt to avoid polluting the output with a stacktrace:

In [ ]:
#TODO: get the consumer
consumer = tt.<FILL>

try:
    #TODO: get message and print it if not null
    <FILL>
except KeyboardInterrupt:
    pass

If you run the previous cell several times you will notice that it always restarts from the beginning by default. We have configured Kafka to persist the data for a quite long time. So be also careful not to leave data producers running uselessly.

This is useful for testing and re-running the same pipeline several time. But in production setting, you may want to remember the last message seen and continue from that. Luckily, Kafka can do it for you if you define a consumer group and commit the current offset (see the documentation: http://pykafka.readthedocs.io/en/latest/usage.html#consumer-patterns ). You can either set the auto_commit_enable, or manually call the commit_offsets method.

**1.e** Rewrite the above consummer loop to only show the newest elements every-time it is run. You can modify and re-run the producing loop to add some more messages.

In [ ]:
from pykafka.common import OffsetType

#TODO: get the consumer at latest offset, use a unique name for the consumer group
consumer_group_name=('test-group-' + username).encode()
consumer = tt.<FILL>

try:
    #TODO: get message and print it if not null
    <FILL>
except KeyboardInterrupt:
    pass

We will now generate a little bit more data and see how to make a live plot. We can use another topic, like `b"test-random-<insert you username>"`.

**1.f** First write an infinite generator (or just a for loop) that outputs the 2D coordinates of a random walk (here you can simply take a random walk on a grid, by taking a random jump of length 1 in any of the 4 directions at each step). Don't forget to add a `time.sleep(1)` each time after you publish it to Kafka.

In [ ]:
tr = client.topics[('test-random-' + username).encode()]

#TODO: import math dependencies

#TODO: create a random walk generator
def <FILL>:
    
#TODO: fill the missing parts to produce messages for the random generated walk.
with tr.get_sync_producer() as producer:
    try:
        for p in <FILL>:
            <FILL>
            time.sleep(0.1)
    except KeyboardInterrupt:
        pass

**1.g** Now we will use the library bokeh ( https://bokeh.pydata.org/en/latest/docs/user_guide.html ) for plotting. It allows for live update of the graph on the notebook, pushing data from the kernel. First you need to create a line plot with a ColumnDataSource initialized with empty lists for x and y. Then, in the consuming loop, you will parse the message and update the data of the source. Make sure your code would work for a long running job without eating all the memory ;-)

In [ ]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Range1d
import time

output_notebook()

x=[]
y=[]
source = ColumnDataSource(data=dict(x=x, y=y))

p = figure(plot_width=850,plot_height=450)
p.line('x', 'y', source=source)

p.x_range = Range1d(-50, 50)
p.y_range = Range1d(-50, 50)

handle=show(p, notebook_handle=True)

#TODO: get a handle on the consumer from test-random-<username>
consumer = <FILL>

#TODO: fill missing parts to get the X,Y pairs, and plot the last 20 values.
try:
    for message in consumer:
        if message is not None:
            
            <FILL>
            
            source.data = dict(x=x, y=y)
            push_notebook(handle=handle)
            time.sleep(0.1)
except KeyboardInterrupt:
    pass


Note that in a notebook you can only run one cell at the time. You thus must stop the producer before you can run the consumer. If you want to play with the producer and the consumer at the same time, feel free to copy one of them to another notebook or in a python script to be able to run both of them simultaneously. Do not forget to include the cells needed initialize the kafka client as well. If you are working with a teammate (e.g. one is producer and the other is the consumer), make sure that you agree on using the same topic name.

## 2. Spark streaming

In the last module, you have experimented with Spark, processing batches of data in RDDs and dataframes. Spark also supports "streaming", namely it defines a stream as a sequence of mini-batches. You can find a pretty detailed introduction here: https://spark.apache.org/docs/2.3.2/streaming-programming-guide.html. Pay particular attention to the section [_points to remember_](https://spark.apache.org/docs/2.3.2/streaming-programming-guide.html#points-to-remember).

In [ ]:
import pyspark
from pyspark.sql import SparkSession
import os

os.environ['PYSPARK_PYTHON'] = '/opt/anaconda3/bin/python'

conf = pyspark.conf.SparkConf()
conf.setMaster('yarn')
conf.setAppName('streaming-{0}'.format(username))
conf.set('spark.executor.memory', '1g')
conf.set('spark.executor.instances', '1')
conf.set('spark.executor.cores', '2') # must be greater than the number of sources
conf.set('spark.port.maxRetries', '100')
#conf.set('spark.jars.packages', 'org.apache.spark:spark-streaming-kafka-0-10_2.11:2.3.2')
conf.set('spark.jars.packages', 'org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.2')
sc = pyspark.SparkContext.getOrCreate(conf)
conf = sc.getConf()
sc

In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

# Create a StreamingContext with two working thread and batch interval of 2 second.
# Each time you stop a StreamingContext, you will need to recreate it here.
ssc = StreamingContext(sc, 2)
ssc.checkpoint('hdfs:///homes/{}/checkpoint/'.format(username))

You can open the Spark UI, following the link in the output above and you should notice a new tab (only when a stream is running), showing the processing of each mini-batch.

### 2.1 From a RDD queue

We will start with a very simple stream, out of a list of RDDs generated manually. Here is an example of RDDs with a quite simple counter. 

```
rdds = []
for i in range(20):
    rdds.append(sc.range(i * 100, (i + 1) * 100))

```

**2.1.a** In the same vein, generate a list of RDDs with random numbers with a uniform distribution. (hint: have a look at the [spark MLlib package](https://spark.apache.org/docs/2.3.2/api/python/pyspark.mllib.html))

In [ ]:
rdds = []
# TODO: create the RDD of random values in range [0,1)
<FILL>

Now let's create the stream and print it from the stream processor. The python API for spark streaming has only a few options to get your results back. `pprint`is the most convenient for debugging and testing, `saveAsTextFiles` can be used for persisting the output and the most generic one is `foreachRDD` which can execute any function. ( https://spark.apache.org/docs/latest/streaming-programming-guide.html#output-operations-on-dstreams )

Once you start the stream, you can check on the UI what is happening and you should see the output here after each batch. You will need to explicitely call stop, which requires you to re-instanciate the streaming context if you want to start a new stream, or re-start this one.

**2.1.b** Run the stream and don't forget to stop it.

In [ ]:
dstream = ssc.queueStream(rdds)

# Print the first ten elements of each RDD generated in this DStream to the console
dstream.pprint()
ssc.start()

In [ ]:
ssc.stop(stopSparkContext=False)

Now let's get back to our random walk example. This demonstrate the capacity of Spark streaming to maintain a current state and apply updates to it. The state is backuped in your user folder on hdfs (as set while creating the streaming context) and you could pass this when recreating the context to re-start from the point when you stopped it.

Do not forget to stop your StreamContext (with `stopSparkContext=False`) when you are done. And remember to recreate the StreamContext after you stop it before reusing it.

**2.1.c** Apply some transformations on the stream of random data such that it outputs 2D coordinates of a random walk.

In [ ]:
# Recreate the streaming context if it has been stopped
ssc = StreamingContext(sc, 2)
ssc.checkpoint('hdfs:///homes/{}/checkpoint/'.format(username))

In [ ]:
import math

def walk(randValue, xyPosition):
    if xyPosition is None:
        xyPosition = (0,0)
    if randValue:
        # TODO: randomly move xyPosition to one of 4 directions and return it
        <FILL>
    else
        return None

In [ ]:
dstream = ssc.queueStream(rdds)
dstream = dstream.map(lambda x: (1, x)).updateStateByKey(walk)

**2.1.d** Modify the following output function such that it produces data that can then by plotted by the same function you wrote before in the previous section.

In [ ]:
from pykafka import KafkaClient

def sendToKafka(iter):
    client = KafkaClient(zookeeper_hosts=ZOOKEEPER_QUORUM)
    with client.topics[('test-random-' + username).encode()].get_sync_producer() as producer:
        for record in iter:
            # TODO
            producer.produce(<FILL>)

In [ ]:
dstream.map(lambda x: len(x[1])).foreachRDD(lambda rdd: rdd.foreachPartition(sendToKafka))

**2.1.e** Now you can start it and while it is running launch the plot cell too. Note that it will stop producing events after all the elements of rdds have been processed. You can stop the StreamContext at that point.

In [ ]:
ssc.start()  # Start the computation

In [ ]:
ssc.stop(stopSparkContext=False) # Don't forget to stop the stream when you are done

### 2.2 From Kafka

Here, we will consume message from the `wiki-edits` topic. It contains the stream of events fetched from the wikimedia sites.
More info: https://www.mediawiki.org/wiki/API:Recent_changes_stream



In [ ]:
from  pyspark.streaming.kafka import KafkaUtils

In [ ]:
ssc = StreamingContext(spark.sparkContext, 10)

In [ ]:
dstream = KafkaUtils.createStream(ssc, ZOOKEEPER_QUORUM, ('test-wiki-group-' + username).encode(), {'wiki-edits': 1})

In [ ]:
dstream.pprint()

In [ ]:
ssc.start()

In [ ]:
ssc.stop(stopSparkContext=False, stopGraceFully=True)

Prototyping spark transformations on a stream can be cumbersome, so we can also create a static RDD from Kafka to experiment first.

In [ ]:
from pyspark.streaming.kafka import OffsetRange

In [ ]:
wiki_rdd_source = KafkaUtils.createRDD(sc, {'bootstrap.servers': 'iccluster042.iccluster.epfl.ch:6667'}, [OffsetRange('wiki-edits',0,10000,10100)])

The code above creates a RDD from the `wiki-edits` Kafka topic, that goes from offset `10000` to offset `10100`.

Note: we start from the 10000th message, but you can play with different offsets.

In [ ]:
wiki_rdd_source.first()

In [ ]:
wiki_rdd_source.count()

We can see that we got the same type of data  as in the stream above.

Now, we can write the function to parse the json messages.

Note: sometimes the messages are invalid json, so we will use `flatMap` to discard those.

In [ ]:
def parse_wiki(doc):
    try:
        return [json.loads(doc)]
    except json.decoder.JSONDecodeError:
        return []

In [ ]:
wiki_rdd = wiki_rdd_source.flatMap(lambda x: parse_wiki(x[1]))

In [ ]:
wiki_rdd.first()

In [ ]:
wiki_rdd.count()

We can also use this on the stream:

In [ ]:
ssc = StreamingContext(spark.sparkContext, 10)
dstream = KafkaUtils.createStream(ssc, ZOOKEEPER_QUORUM, ('test-wiki-group-' + username).encode(), {'wiki-edits': 1})

In [ ]:
wiki_dstream = dstream.flatMap(lambda x: parse_wiki(x[1]))

In [ ]:
wiki_dstream.pprint()

In [ ]:
ssc.start()

In [ ]:
ssc.stop(stopSparkContext=False, stopGraceFully=True)

**2.2.a** Create a stream from `wiki-edits` which contains only the `type` and `wiki` fields.

## 2.3 Window operations

We have seen how to collect data into a stream on spark RDDs and how to apply simple transformations on them. Spark allows to perform window operations on streams.

Documentation: https://spark.apache.org/docs/2.3.0/streaming-programming-guide.html#window-operations

Let's have a look at what we can get from the `wiki-edits`. We will perform operations on windows of size 10 minutes, every 2 seconds.

As a simple example, here is how to count the number of messages for such a window.

In [ ]:
ssc = StreamingContext(spark.sparkContext, 2)
ssc.checkpoint('/tmp/chk')
dstream = KafkaUtils.createStream(ssc, ZOOKEEPER_QUORUM, ('test-wiki-group-' + username).encode(), {'wiki-edits': 1})
wiki_dstream = dstream.flatMap(lambda x: parse_wiki(x[1]))

In [ ]:
# 600s = 10minutes, every 2 seconds
wiki_counts = wiki_dstream.countByWindow(600, 2)

In [ ]:
wiki_counts.pprint()

In [ ]:
ssc.start()

Look at the documentation to see how to use the window operations.

**2.3.a** Create a stream that counts the actions done by a bot and those done by humans within the time window.

**2.3.b** Create a stream that gets the number of actions by `type`. Also get the number of actions by `wiki`.

**2.3.c** Send the above streams to kafka topics (example in **2.1**) and use that to dynamically plot:
- the volume of actions by bots vs actions by humans.
- the volume of actions by `type`, by `wiki`,

Use `wiki-plot-bot-<username>`,  `wiki-plot-type-<username>` and `wiki-plot-wiki-<username>` for the respective topic names.

In [ ]:
#TODO 2.3.a: create a stream and count actions by bot vs actions by humans

In [ ]:
#TODO 2.3.b: Create stream that gets the number of actions by type. Also get the number of actions by wiki.

In [ ]:
#TODO 2.3.c: write the streams to topics

In [ ]:
#TODO: create the plot in bokeh

In [ ]:
#TODO: stop the stream context, and release the spark context to free resources